# Problem Statement :- There are six classes like Building, Forest, Glacier mountain, Sea, Streets. And we need to make a model such that it should predict image and tell what class it belongs to.

## For this problem statement there are various appraoches we can use, but as per constraint we cannot use CNN and RNN. If we use we can get furher strong model to utilsize for our project

### For this we will get meta-data firstly

In [39]:
import os
import pandas as pd

# Define the image folder path
image_folder = "/home/sagar/Bombay Software/dataset_full"

# Initialize lists to store data for CSV
data = {'Folder_Image': [], 'Image_Path': [], 'Image_Type': [], 'Image_name': []}

# Iterate through the folders and images
for folder_name in os.listdir(image_folder):
    folder_path = os.path.join(image_folder, folder_name)
    if os.path.isdir(folder_path):
        for image_name in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_name)
            if os.path.isfile(image_path):
                data['Folder_Image'].append(folder_name)
                data['Image_Path'].append(image_path)
                data['Image_Type'].append(folder_name)
                data['Image_name'].append(image_name)# Assuming folder name is the image type

# Create DataFrame from the data
df = pd.DataFrame(data)

# Save DataFrame to CSV
csv_file_path = "meta_data_added_original.csv"  # Update with your desired CSV file path
df.to_csv(csv_file_path, index=False)


### Above we created meta-data of original data by folder by folder in a single Csv file

In [40]:
df.head(1)

,Folder_Image,Image_Path,Image_Type,Image_name
0,Streets,/home/sagar/Bombay Software/dataset_full/Stree...,Streets,12319.jpg


In [37]:
df2 = pd.read_csv("/home/sagar/Bombay Software/meta_data_original.csv")
df2["Image_Type"].value_counts()

Forest       2745
Streets       501
Mountains     501
Glacier       501
Building      501
Sea           501
Name: Image_Type, dtype: int64

### As we can see above Forest is at 1st position because it have 2745 datapoints which will make bais of model to forest most of the time, beacause other classes are less in datapoint count i.e approx. 500 datapoints for other remaining classes

### So we can upsample less count data as mentioned in assignment question to use feature augmentations techniques
###
### Assignment Components
### 1. Data download and preprocessing.
### 2. Feature extraction : You need to take multiple sets of handcrafted features, keep 3
### features at least in a feature set. The features can be Low-level Vision: Histogram and
### Histogram equalisation, Gray-scale transformation, Image Smoothing, Connected
### components in images. Mid-level Vision: Edge Detection using Gradients, Sobel,
### Canny; Line detection using Hough transforms; Semantic information using RANSAC;
### Image region descriptor using SIFT etc.
### 3. Dimensionality reduction if needed.
### 4. Classification algorithm of your choice with explanation.
### 5. Evaluation components.
### 6. Flask app: One should be able to upload an image and get the classification result.

In [32]:
import os
import cv2

# Define the input folder containing the original images
input_folder = "/home/sagar/Bombay Software/dataset_full (copy)/Streets"

# Define the output folder to store the feature images
output_folder = input_folder

# Function to extract features and save feature images
def extract_and_save_feature_images(image_path, feature_name):
    # Read the image
    image = cv2.imread(image_path)
    
    # Apply the desired feature extraction technique based on feature_name
    if feature_name == "histogram":
        feature_image = cv2.calcHist([image], [0], None, [256], [0, 256])  # Example: Histogram
    elif feature_name == "Equilize_histogram":  # Example: Histogram
        image = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
        feature_image = cv2.equalizeHist(image)
    elif feature_name == "smoothed_image":
        feature_image = cv2.GaussianBlur(image, (5, 5), 0)  # Example: Canny Edge Detection
    elif feature_name == "gray_image":
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        feature_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    elif feature_name == "edge_detection_canny":
        feature_image = cv2.Canny(image, 100, 200)
    
    # Save the feature image
    feature_image_path = os.path.join(output_folder, f"{os.path.basename(image_path)}_{feature_name}.jpg")
    cv2.imwrite(feature_image_path, feature_image)

# Iterate through the input folder and process each image
for image_name in os.listdir(input_folder):
    image_path = os.path.join(input_folder, image_name)
    if os.path.isfile(image_path):
        # Extract and save features for each image
        extract_and_save_feature_images(image_path, "histogram")
        extract_and_save_feature_images(image_path, "Equilize_histogram")
        extract_and_save_feature_images(image_path, "smoothed_image")
        extract_and_save_feature_images(image_path, "gray_image")
        extract_and_save_feature_images(image_path, "edge_detection_canny")
        # Add more calls to extract_and_save_feature_images for other features

print("Feature extraction and saving complete.")


Feature extraction and saving complete.


### Above we have used Equilize_histogram, smoothed_image, gray_image, edge_detection_canny technique which covered low and mid level feature engineering and atleast 3 were asked I considered 4 , histogram dont play significance role in getting features so ignored in below part for training and model building

In [35]:
df = pd.read_csv("/home/sagar/Bombay Software/meta_data_added_features.csv")

In [36]:
df["Image_Type"].value_counts()

Forest       16470
Streets       3000
Mountains     3000
Glacier       3000
Building      3000
Sea           3000
Name: Image_Type, dtype: int64

### Now we got upsampled data from 500 to 3k points by low and mid level of feature engineering

In [47]:
def extract_hog_features(image_path):
    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hog = cv2.HOGDescriptor()
    hog_features = hog.compute(gray_image)
    return hog_features.flatten()

hog_features = extract_hog_features("/home/sagar/Bombay Software/dataset_full (copy)/Mountains/10460.jpg_gray_image.jpg")

In [48]:
hog_features

array([0.00286013, 0.00626544, 0.14861314, ..., 0.24073915, 0.24073915,
       0.16249195], dtype=float32)

### Above we are considering HOGDescriptor() from opencv for feature extraction from images

In [59]:
import cv2
import os
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

# Function to extract HOG features from an image
def extract_hog_features(image_path):
    try:
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error loading image: {image_path}")
            return None
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        hog = cv2.HOGDescriptor()
        hog_features = hog.compute(gray_image)
        if hog_features is None:
            print(f"Error computing HOG features for image: {image_path}")
            return None
        return hog_features.flatten()
    except Exception as e:
        print(f"Error processing image: {image_path} - {e}")
        return None


# Define paths to your image data
image_folder = "/home/sagar/Bombay Software/dataset_full (copy)"
label_file = "/home/sagar/Bombay Software/meta_data_added_features.csv"

# Load labels from CSV file
labels_df = pd.read_csv(label_file)
# labels_dict = dict(zip(labels_df['Image_Path'], labels_df['Image_Type']))

# Initialize lists to store features and labels
features = []
labels = []

for ind in labels_df.index:
    hog_features = extract_hog_features(labels_df["Image_Path"][ind])
    features.append(hog_features)



# Extract features from images
# for image_name in os.listdir(image_folder):
#     image_path = os.path.join(image_folder, image_name)
#     if os.path.isfile(image_path):
#         # Extract HOG features
#         hog_features = extract_hog_features(image_path)
#         features.append(hog_features)
# #         labels.append(labels_dict[image_name])

# # Convert lists to numpy arrays
# X = np.array(features)
# y = np.array(labels)

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Initialize Random Forest classifier
# rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# # Train the classifier
# rf_classifier.fit(X_train, y_train)

# # Make predictions on the test set
# y_pred = rf_classifier.predict(X_test)

# # Evaluate the classifier
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy:.2f}")

# # Print classification report
# print(classification_report(y_test, y_pred))


Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/16906.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/23389.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/10691.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/12762.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/18705.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/15939.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/17761.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Stre

Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/11583.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/9160.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/3546.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/23182.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/18763.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/1030.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/13317.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets

Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/5026.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/23795.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/10503.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/931.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/23633.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/18006.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/16072.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets

Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/14280.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/20259.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/15339.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/18517.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/2788.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/16652.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/18383.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Stree

Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/7559.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/6820.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/21749.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/9720.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/6240.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/23014.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/16682.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Streets/

...

Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/6286.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/8882.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/4322.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/9464.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/673.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/23093.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/12929.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/5678.jpg_histogram.jpg - vector::_

Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/15703.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/13439.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/5154.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/3327.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/19378.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/23965.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/10686.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/10183.jpg_histogram.jpg - vect

Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/17695.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/19371.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/10071.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/18103.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/19367.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/7901.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/5603.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/4452.jpg_histogram.jpg - vecto

Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/21936.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/8203.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/18347.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/6226.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/625.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/299.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/2671.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/17788.jpg_histogram.jpg - vector::_

Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/3074.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/16752.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/2740.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/2806.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/10529.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/21083.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/6319.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (copy)/Sea/1027.jpg_histogram.jpg - vector:

In [61]:
len(features)

31470

In [62]:
labels_df.shape

(31470, 3)

In [71]:
labels_df["features_from_images"][0]

array([0.17831197, 0.03282047, 0.01394701, ..., 0.3249025 , 0.14251456,
       0.03947674], dtype=float32)

In [64]:
labels_df["features_from_images"] = features

### Above we extracted features and added to new meta-data of upsampled csv file

In [65]:
labels_df.head(1)

,Folder_Image,Image_Path,Image_Type,features_from_images
0,Streets,/home/sagar/Bombay Software/dataset_full (copy...,Streets,"[0.17831197, 0.03282047, 0.01394701, 0.0301816..."


In [66]:
labels_df.to_csv("Processed_data_with_feature_extracted.csv",index = False)

In [7]:
import pandas as pd
labels_df = pd.read_csv("Processed_data_with_feature_extracted.csv")

In [8]:
labels_df.shape

(31470, 4)

In [9]:
labels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31470 entries, 0 to 31469
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Folder_Image          31470 non-null  object
 1   Image_Path            31470 non-null  object
 2   Image_Type            31470 non-null  object
 3   features_from_images  26190 non-null  object
dtypes: object(4)
memory usage: 983.6+ KB


### Above we can see "features_from_images" have some None value which is because histogram dont give feature value as I stated earlier as well so we will ignore those data rows

In [10]:
labels_dfx = labels_df[labels_df["features_from_images"].notnull()]

In [11]:
labels_dfx.shape

(26190, 4)

In [12]:
labels_dfx.head(1)

,Folder_Image,Image_Path,Image_Type,features_from_images
0,Streets,/home/sagar/Bombay Software/dataset_full (copy...,Streets,[0.17831197 0.03282047 0.01394701 ... 0.324902...


In [13]:
labels_dfx["Image_Type"].value_counts()

Forest       13700
Streets       2500
Glacier       2500
Building      2500
Mountains     2495
Sea           2495
Name: Image_Type, dtype: int64

In [14]:
labels_dfx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26190 entries, 0 to 31469
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Folder_Image          26190 non-null  object
 1   Image_Path            26190 non-null  object
 2   Image_Type            26190 non-null  object
 3   features_from_images  26190 non-null  object
dtypes: object(4)
memory usage: 1023.0+ KB


In [15]:
labels_dfx.head(1)

,Folder_Image,Image_Path,Image_Type,features_from_images
0,Streets,/home/sagar/Bombay Software/dataset_full (copy...,Streets,[0.17831197 0.03282047 0.01394701 ... 0.324902...


In [16]:
import re

# Define a function to extract and convert numerical values
def extract_numbers(s):
    numbers = re.findall(r'\d+\.\d+', s)
    return [float(num) for num in numbers]

# Apply the function to the 'features' column and overwrite it
labels_dfx["features_from_images"] = labels_dfx["features_from_images"].apply(lambda x: extract_numbers(x[1:-1]))


/tmp/ipykernel_7199/3773274554.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels_dfx["features_from_images"] = labels_dfx["features_from_images"].apply(lambda x: extract_numbers(x[1:-1]))


### Above we are getting numeric values from array features and above function will also help to tackle unwanted text in array features

In [17]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

X = labels_dfx["features_from_images"] # Update with your features


y = labels_dfx["Image_Type"]  # Update with your target variable
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)



### Now we are done with our data sampling and depedent, independent data as it was multi-class classification problem so dependent variable i.e 'y' will be Image_type

#### Approaches to be consider :- As we have contraint we can only use ML Algorithm for this project
#### So we will use KNN , SVM, RandomForest and essemble techniques
#### We will 1st try with all data given by your side then we will use equalization of data

In [14]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report

rf_classifier = RandomForestClassifier()

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Perform grid search with 5-fold cross-validation
grid_search = GridSearchCV(rf_classifier, param_grid, cv=5, n_jobs=-1, verbose=1)


#### We will use Smote and Ros for sampling of data "Oversampling works by replicating samples of the minority class through several variations of the oversampling technique, such as random oversampling (ROS), synthetic minority oversampling (SMOTE)"

In [20]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)

In [21]:
from imblearn.over_sampling import SMOTE
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
smote = SMOTE(random_state=42)
X_train_flat = np.array([label for sublist in X_train for label in sublist])
X_train_flat = X_train_flat.reshape(-1, 1)
X_train_flat = X_train_flat[:len(y_train)]
X_train_resampled, y_train_resampled = ros.fit_resample(X_train_flat, y_train)

In [16]:
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best hyperparameters from grid search
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

Fitting 5 folds for each of 243 candidates, totalling 1215 fits


/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

...

/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

Best Hyperparameters: {'max_depth': 20, 'max_features': 'auto', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 300}


In [17]:
print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'max_depth': 20, 'max_features': 'auto', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 300}


In [18]:
best_rf_model = grid_search.best_estimator_

In [6]:
import joblib

# Save the best model obtained from grid search
joblib.dump(best_rf_model, 'best_random_forest_model.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')


In [43]:
# best_rf_model = grid_search.best_estimator_
X_test_flat = np.array([label for sublist in X_test for label in sublist])
X_test_flat = X_test_flat.reshape(-1, 1)
X_test_flat = X_test_flat[:len(y_test)]

y_pred = best_rf_model.predict(X_test_flat)

# Evaluate the best model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.15979381443298968
Classification Report:
              precision    recall  f1-score   support

           0       0.11      0.20      0.14       516
           1       0.51      0.14      0.22      2760
           2       0.10      0.17      0.13       481
           3       0.11      0.20      0.14       521
           4       0.09      0.18      0.12       474
           5       0.09      0.16      0.12       486

    accuracy                           0.16      5238
   macro avg       0.17      0.17      0.14      5238
weighted avg       0.32      0.16      0.18      5238



#### Above we are getting 16 % accuracy on test data which is low

In [47]:
import numpy as np
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# svm_classifier = SVC(kernel='rbf')
knn_classifier = KNeighborsClassifier(n_neighbors=2)

# Train the SVM classifier
knn_classifier.fit(X_train_resampled, y_train_resampled)
# svm_classifier.fit(X_train_resampled, y_train_resampled)

# Make predictions on the test data
# y_pred = svm_classifier.predict(X_test)


KNeighborsClassifier(n_neighbors=2)

In [48]:
y_pred = knn_classifier.predict(X_test_flat)

# Evaluate the best model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.3035509736540664
Classification Report:
              precision    recall  f1-score   support

           0       0.09      0.14      0.11       516
           1       0.53      0.48      0.50      2760
           2       0.10      0.11      0.10       481
           3       0.11      0.11      0.11       521
           4       0.09      0.09      0.09       474
           5       0.10      0.10      0.10       486

    accuracy                           0.30      5238
   macro avg       0.17      0.17      0.17      5238
weighted avg       0.33      0.30      0.31      5238



#### Above we are getting 30 % accuracy on test data

In [22]:
import numpy as np
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

svm_classifier = SVC(kernel='rbf')

# Train the SVM classifier
svm_classifier.fit(X_train_resampled, y_train_resampled)


SVC()

In [24]:
X_test_flat = np.array([label for sublist in X_test for label in sublist])
X_test_flat = X_test_flat.reshape(-1, 1)
X_test_flat = X_test_flat[:len(y_test)]

y_pred = svm_classifier.predict(X_test_flat)

# Evaluate the best model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.09851088201603665
Classification Report:
              precision    recall  f1-score   support

           0       0.09      0.25      0.14       516
           1       0.67      0.00      0.00      2760
           2       0.07      0.02      0.03       481
           3       0.10      0.51      0.16       521
           4       0.11      0.21      0.14       474
           5       0.11      0.03      0.05       486

    accuracy                           0.10      5238
   macro avg       0.19      0.17      0.09      5238
weighted avg       0.40      0.10      0.05      5238



#### Above we are getting 10 % accuracy on test data

#### Now we will try random forest with smote sampling

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
smote = SMOTE(random_state=42)
X_train_flat = np.array([label for sublist in X_train for label in sublist])
X_train_flat = X_train_flat.reshape(-1, 1)
X_train_flat = X_train_flat[:len(y_train)]
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_flat, y_train)


In [21]:
X_train_flat = X_train_flat[:len(y_train)]
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best hyperparameters from grid search
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

Fitting 5 folds for each of 243 candidates, totalling 1215 fits


/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

...

/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Best Hyperparameters: {'max_depth': 10, 'max_features': 'auto', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}


/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

In [22]:
print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'max_depth': 10, 'max_features': 'auto', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}


In [25]:
best_rf_model = grid_search.best_estimator_
X_test_flat = np.array([label for sublist in X_test for label in sublist])
X_test_flat = X_test_flat.reshape(-1, 1)
X_test_flat = X_test_flat[:len(y_test)]

y_pred = best_rf_model.predict(X_test_flat)

# Evaluate the best model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.5269186712485682
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       516
           1       0.53      1.00      0.69      2760
           2       0.00      0.00      0.00       481
           3       0.00      0.00      0.00       521
           4       0.00      0.00      0.00       474
           5       0.00      0.00      0.00       486

    accuracy                           0.53      5238
   macro avg       0.09      0.17      0.12      5238
weighted avg       0.28      0.53      0.36      5238



/home/sagar/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sagar/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sagar/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Above we are getting 53 % accuracy .But as I said its baised for forest image type mostly

In [26]:
from imblearn.over_sampling import SMOTE
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load your image data and metadata into pandas DataFrames
# image_data = pd.read_csv("path_to_image_data.csv")  # Update with your actual path
# metadata = pd.read_csv("path_to_metadata.csv")  # Update with your actual path

# Merge image data and metadata based on a common column (e.g., image ID)
# merged_data = pd.merge(image_data, metadata, on="image_id")

# merged_data = pd.read_csv("/home/sagar/Bombay Software/meta_data_added_features.csv")
# # Define features (X) and target variable (y)
X = labels_dfx["features_from_images"] # Update with your features
# X = X_train_reshaped

y = labels_dfx["Image_Type"]  # Update with your target variable
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
smote = SMOTE(random_state=42)
X_train_flat = np.array([label for sublist in X_train for label in sublist])
X_train_flat = X_train_flat.reshape(-1, 1)
X_train_flat = X_train_flat[:len(y_train)]
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_flat, y_train)


import numpy as np
# X_train_flat = np.array([label for sublist in X_train_resampled for label in sublist])
# X_train_flat = X_train_flat.reshape(-1, 1)

rf_classifier = RandomForestClassifier(class_weight='balanced', random_state=42)

# Evaluate the model on the test set


In [28]:
rf_classifier.fit(X_train_resampled, y_train_resampled)

RandomForestClassifier(class_weight='balanced', random_state=42)

In [29]:
X_test_flat = np.array([label for sublist in X_test for label in sublist])
X_test_flat = X_test_flat.reshape(-1, 1)
X_test_flat = X_test_flat[:len(y_test)]
y_pred = rf_classifier.predict(X_test_flat)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Print classification report
print(classification_report(y_test, y_pred))

Accuracy: 0.19
              precision    recall  f1-score   support

           0       0.11      0.19      0.14       516
           1       0.54      0.20      0.29      2760
           2       0.10      0.16      0.12       481
           3       0.12      0.19      0.15       521
           4       0.08      0.15      0.11       474
           5       0.09      0.16      0.12       486

    accuracy                           0.19      5238
   macro avg       0.17      0.18      0.15      5238
weighted avg       0.33      0.19      0.21      5238



#### Above we used Random Forest with class weight balance technique which giving 19 % accuracy

In [77]:
import cv2
import numpy as np

# Load and preprocess the test image
test_image_path = "/home/sagar/Bombay Software/dataset_full (copy)/Mountains/194.jpg"
# test_image = cv2.imread(test_image_path)
# if test_image is None:
#     print(f"Error loading test image: {test_image_path}")
#     exit(1)

def extract_hog_features(image_path):
    try:
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error loading image: {image_path}")
            return None
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        hog = cv2.HOGDescriptor()
        hog_features = hog.compute(gray_image)
        if hog_features is None:
            print(f"Error computing HOG features for image: {image_path}")
            return None
        return hog_features.flatten()
    except Exception as e:
        print(f"Error processing image: {image_path} - {e}")
        return None

test_image = extract_hog_features(test_image_path)

# import re

# # Define a function to extract and convert numerical values
# def extract_numbers_vectorized(array):
#     return np.array([[float(num) for num in re.findall(r'\d+\.\d+', s)] for s in array])


# # Apply the function to the 'features' column and overwrite it
# # test_image = test_image.decode('utf-8')
# # test_image = extract_numbers_vectorized(test_image)


# # import numpy as np
# # test_image = np.array([label for sublist in test_image for label in sublist])


# # prediction = rf_classifier.predict(test_image.reshape(1, -1))

# # # Print the predicted class
# # print("Predicted class:", prediction)



In [78]:
test_image = test_image.reshape(-1, 1)

In [79]:
test_image[:1]

array([[0.05296015]], dtype=float32)

In [80]:
y_test = 1

In [81]:
test_image = test_image[:y_test]

In [82]:
test_image

array([[0.05296015]], dtype=float32)

In [83]:
prediction = rf_classifier.predict(test_image.reshape(1, -1))

# # Print the predicted class
print("Predicted class:", prediction)


Predicted class: [1]


In [88]:
decoded_labels = label_encoder.inverse_transform(prediction)

print(decoded_labels)  # Output: ['dog' 'cat' 'bird' 'dog']

['Forest']


## Experiment 2

#### Now we will use data equalization technique where all classes will be of equal length

In [3]:
import os
import random
import shutil

# Path to the directory containing the files
directory_path = '/home/sagar/Bombay Software/dataset_full (another copy)/Forest'

# Get a list of all files in the directory
all_files = os.listdir(directory_path)

# Randomly select 250 files from the list
random_files = random.sample(all_files, 500)

# Destination directory to move the selected files
destination_directory = '/home/sagar/Bombay Software/dataset_full (another copy)/forest'

# Create the destination directory if it doesn't exist
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)

# Move the selected files to the destination directory
for file_name in random_files:
    source_file = os.path.join(directory_path, file_name)
    destination_file = os.path.join(destination_directory, file_name)
    shutil.move(source_file, destination_file)


#### Above we are downsampling forest data points only as it was only high in count in actual shared dataset

In [4]:
import os
import pandas as pd

# Define the image folder path
image_folder = "/home/sagar/Bombay Software/dataset_full (another copy)"

# Initialize lists to store data for CSV
data = {'Folder_Image': [], 'Image_Path': [], 'Image_Type': [], 'Image_name': []}

# Iterate through the folders and images
for folder_name in os.listdir(image_folder):
    folder_path = os.path.join(image_folder, folder_name)
    if os.path.isdir(folder_path):
        for image_name in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_name)
            if os.path.isfile(image_path):
                data['Folder_Image'].append(folder_name)
                data['Image_Path'].append(image_path)
                data['Image_Type'].append(folder_name)
                data['Image_name'].append(image_name)# Assuming folder name is the image type

# Create DataFrame from the data
df = pd.DataFrame(data)

# Save DataFrame to CSV
csv_file_path = "meta_data_chages_made.csv"  # Update with your desired CSV file path
df.to_csv(csv_file_path, index=False)


#### Again we created meta-data of original dataset for new datapoints

In [5]:
df.head(1)

,Folder_Image,Image_Path,Image_Type,Image_name
0,Streets,/home/sagar/Bombay Software/dataset_full (anot...,Streets,12319.jpg


In [6]:
df["Image_Type"].value_counts()

Streets      500
forest       500
Mountains    500
Glacier      500
Building     500
Sea          500
Name: Image_Type, dtype: int64

#### See now all classes have equal no. of datapoints in it, now we will do same above approaches for this current dataset

In [12]:
import os
import cv2

# Define the input folder containing the original images
input_folder = "/home/sagar/Bombay Software/dataset_full (another copy)/Streets"

# Define the output folder to store the feature images
output_folder = input_folder

# Function to extract features and save feature images
def extract_and_save_feature_images(image_path, feature_name):
    # Read the image
    image = cv2.imread(image_path)
    
    # Apply the desired feature extraction technique based on feature_name
    if feature_name == "histogram":
        feature_image = cv2.calcHist([image], [0], None, [256], [0, 256])  # Example: Histogram
    elif feature_name == "Equilize_histogram":  # Example: Histogram
        image = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
        feature_image = cv2.equalizeHist(image)
    elif feature_name == "smoothed_image":
        feature_image = cv2.GaussianBlur(image, (5, 5), 0)  # Example: Canny Edge Detection
    elif feature_name == "gray_image":
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        feature_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    elif feature_name == "edge_detection_canny":
        feature_image = cv2.Canny(image, 100, 200)
    
    # Save the feature image
    feature_image_path = os.path.join(output_folder, f"{os.path.basename(image_path)}_{feature_name}.jpg")
    cv2.imwrite(feature_image_path, feature_image)

# Iterate through the input folder and process each image
for image_name in os.listdir(input_folder):
    image_path = os.path.join(input_folder, image_name)
    if os.path.isfile(image_path):
        # Extract and save features for each image
        extract_and_save_feature_images(image_path, "histogram")
        extract_and_save_feature_images(image_path, "Equilize_histogram")
        extract_and_save_feature_images(image_path, "smoothed_image")
        extract_and_save_feature_images(image_path, "gray_image")
        extract_and_save_feature_images(image_path, "edge_detection_canny")
        # Add more calls to extract_and_save_feature_images for other features

print("Feature extraction and saving complete.")


Feature extraction and saving complete.


#### Again we upsampled and created new metadata of csv

In [13]:
import os
import pandas as pd

# Define the image folder path
image_folder = "/home/sagar/Bombay Software/dataset_full (another copy)"

# Initialize lists to store data for CSV
data = {'Folder_Image': [], 'Image_Path': [], 'Image_Type': [], 'Image_name': []}

# Iterate through the folders and images
for folder_name in os.listdir(image_folder):
    folder_path = os.path.join(image_folder, folder_name)
    if os.path.isdir(folder_path):
        for image_name in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_name)
            if os.path.isfile(image_path):
                data['Folder_Image'].append(folder_name)
                data['Image_Path'].append(image_path)
                data['Image_Type'].append(folder_name)
                data['Image_name'].append(image_name)# Assuming folder name is the image type

# Create DataFrame from the data
df = pd.DataFrame(data)

# Save DataFrame to CSV
csv_file_path = "meta_data_chages_made_sampled.csv"  # Update with your desired CSV file path
df.to_csv(csv_file_path, index=False)


In [14]:
import cv2
import os
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

# Function to extract HOG features from an image
def extract_hog_features(image_path):
    try:
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error loading image: {image_path}")
            return None
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        hog = cv2.HOGDescriptor()
        hog_features = hog.compute(gray_image)
        if hog_features is None:
            print(f"Error computing HOG features for image: {image_path}")
            return None
        return hog_features.flatten()
    except Exception as e:
        print(f"Error processing image: {image_path} - {e}")
        return None


# Define paths to your image data
image_folder = "/home/sagar/Bombay Software/dataset_full (another copy)"
label_file = "/home/sagar/Bombay Software/meta_data_chages_made_sampled.csv"

# Load labels from CSV file
labels_df = pd.read_csv(label_file)
# labels_dict = dict(zip(labels_df['Image_Path'], labels_df['Image_Type']))

# Initialize lists to store features and labels
features = []
labels = []

for ind in labels_df.index:
    hog_features = extract_hog_features(labels_df["Image_Path"][ind])
    features.append(hog_features)

labels_df["features_from_images"] = features



Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Streets/16906.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Streets/23389.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Streets/10691.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Streets/12762.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Streets/18705.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Streets/15939.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Streets/17761.jpg_histogram.jpg - vector::_M_default_append
Error processing ima

Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Streets/19145.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Streets/14312.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Streets/11583.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Streets/9160.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Streets/3546.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Streets/23182.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Streets/18763.jpg_histogram.jpg - vector::_M_default_append
Error processing image

Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Streets/10503.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Streets/931.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Streets/23633.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Streets/18006.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Streets/16072.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Streets/10600.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Streets/2327.jpg_histogram.jpg - vector::_M_default_append
Error processing image:

Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Streets/22109.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Streets/18534.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Streets/24040.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Streets/4963.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Streets/2169.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Streets/12455.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Streets/2946.jpg_histogram.jpg - vector::_M_default_append
Error processing image:

Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Streets/1033.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Streets/1186.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Streets/14227.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Streets/3205.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Streets/5797.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Streets/17576.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Streets/9482.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /

...

Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Sea/19378.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Sea/23965.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Sea/10686.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Sea/10183.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Sea/3266.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Sea/12485.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Sea/20114.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Softwa

Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Sea/7901.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Sea/5603.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Sea/4452.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Sea/7234.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Sea/4257.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Sea/6947.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Sea/12892.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/da

Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Sea/17794.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Sea/24034.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Sea/2086.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Sea/9648.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Sea/17509.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Sea/17760.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Sea/13152.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Softwar

Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Sea/4386.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Sea/21393.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Sea/16511.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Sea/23679.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Sea/21334.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Sea/16136.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Sea/13898.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Softwa

Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Sea/18146.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Sea/9514.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Sea/4800.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Sea/11133.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Sea/15618.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Sea/11238.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Software/dataset_full (another copy)/Sea/13484.jpg_histogram.jpg - vector::_M_default_append
Error processing image: /home/sagar/Bombay Softwar

In [15]:
labels_df.to_csv("Processed_data_with_feature_extracted_sampled.csv",index = False)

In [24]:
labels_df["Image_Type"].value_counts()

Streets      3000
forest       3000
Mountains    3000
Glacier      3000
Building     3000
Sea          3000
Name: Image_Type, dtype: int64

#### Above all classes have equal datapoints in it

In [1]:
import pandas as pd
labels_df = pd.read_csv("Processed_data_with_feature_extracted_sampled.csv")

In [2]:
labels_dfx = labels_df[labels_df["features_from_images"].notnull()]

In [3]:
labels_dfx.shape

(14985, 5)

In [4]:
labels_dfx["Image_Type"].value_counts()

Streets      2500
Glacier      2500
Building     2500
forest       2495
Mountains    2495
Sea          2495
Name: Image_Type, dtype: int64

In [5]:
labels_dfx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14985 entries, 0 to 17999
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Folder_Image          14985 non-null  object
 1   Image_Path            14985 non-null  object
 2   Image_Type            14985 non-null  object
 3   Image_name            14985 non-null  object
 4   features_from_images  14985 non-null  object
dtypes: object(5)
memory usage: 702.4+ KB


In [6]:
labels_dfx.head(1)

,Folder_Image,Image_Path,Image_Type,Image_name,features_from_images
0,Streets,/home/sagar/Bombay Software/dataset_full (anot...,Streets,8949.jpg_Equilize_histogram.jpg,[0.17831197 0.03282047 0.01394701 ... 0.324902...


In [7]:
import re

# Define a function to extract and convert numerical values
def extract_numbers(s):
    if isinstance(s, bytes):
        s = s.decode('utf-8')  # Decode bytes-like object to string if needed
    numbers = re.findall(r'\d+\.\d+', s)
    return [float(num) for num in numbers]


# Apply the function to the 'features' column and overwrite it
# labels_dfx["features_from_images"] = labels_dfx["features_from_images"].apply(lambda x: extract_numbers(x[1:-1]))
labels_dfx["features_from_images"] = labels_dfx["features_from_images"].apply(extract_numbers)



/tmp/ipykernel_23130/1701089855.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels_dfx["features_from_images"] = labels_dfx["features_from_images"].apply(extract_numbers)


In [9]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report

X = labels_dfx["features_from_images"] # Update with your features
# X = X_train_reshaped

y = labels_dfx["Image_Type"]  # Update with your target variable
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
X_train_flat = np.array([label for sublist in X_train for label in sublist])
X_train_flat = X_train_flat.reshape(-1, 1)
X_train_flat = X_train_flat[:len(y_train)]
# X_train_resampled, y_train_resampled = smote.fit_resample(X_train_flat, y_train)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train_flat, y_train)

In [42]:
X_train_flat = np.array([label for sublist in X_train for label in sublist])
X_train_flat = X_train_flat.reshape(-1, 1)
X_train_flat = X_train_flat[:len(y_train)]


knn_classifier = KNeighborsClassifier(n_neighbors=3)

knn_classifier.fit(X_train_resampled, y_train_resampled)



KNeighborsClassifier(n_neighbors=3)

In [43]:
X_test_flat = np.array([label for sublist in X_test for label in sublist])
X_test_flat = X_test_flat.reshape(-1, 1)
X_test_flat = X_test_flat[:len(y_test)]

y_pred = knn_classifier.predict(X_test_flat)

# Evaluate the best model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.18151484818151484
Classification Report:
              precision    recall  f1-score   support

           0       0.17      0.39      0.24       465
           1       0.18      0.24      0.21       554
           2       0.18      0.18      0.18       480
           3       0.16      0.10      0.13       465
           4       0.20      0.09      0.13       502
           5       0.23      0.09      0.13       531

    accuracy                           0.18      2997
   macro avg       0.19      0.18      0.17      2997
weighted avg       0.19      0.18      0.17      2997



#### Above we are getting 18 % accuracy with equal data

In [20]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Initialize base models
rf_classifier = RandomForestClassifier()
svm_classifier = SVC(probability=True)
knn_classifier = KNeighborsClassifier()

# Create a voting ensemble
voting_ensemble = VotingClassifier(estimators=[
    ('random_forest', rf_classifier),
    ('svm', svm_classifier),
    ('knn', knn_classifier)
], voting='soft')  # Use 'soft' for probabilities-based voting

# Train the voting ensemble
voting_ensemble.fit(X_train_resampled, y_train_resampled)

# Make predictions
y_pred = voting_ensemble.predict(X_test_flat)

# Evaluate the ensemble
accuracy = accuracy_score(y_test, y_pred)
print("Ensemble Accuracy:", accuracy)


Ensemble Accuracy: 0.16216216216216217


#### Above esemble technique gives 16 % accuracy which is again low for our deployment

In [21]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report

rf_classifier = RandomForestClassifier()

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Perform grid search with 5-fold cross-validation
grid_search = GridSearchCV(rf_classifier, param_grid, cv=5, n_jobs=-1, verbose=1)


# Fit the grid search to the training data


In [22]:
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best hyperparameters from grid search
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

Fitting 5 folds for each of 243 candidates, totalling 1215 fits


/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

Best Hyperparameters: {'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


In [23]:
print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


In [25]:
best_rf_model = grid_search.best_estimator_
X_test_flat = np.array([label for sublist in X_test for label in sublist])
X_test_flat = X_test_flat.reshape(-1, 1)
X_test_flat = X_test_flat[:len(y_test)]

y_pred = best_rf_model.predict(X_test_flat)

# Evaluate the best model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.1598264931598265
Classification Report:
              precision    recall  f1-score   support

           0       0.13      0.14      0.13       465
           1       0.19      0.17      0.18       554
           2       0.16      0.16      0.16       480
           3       0.15      0.17      0.16       465
           4       0.17      0.19      0.18       502
           5       0.16      0.14      0.15       531

    accuracy                           0.16      2997
   macro avg       0.16      0.16      0.16      2997
weighted avg       0.16      0.16      0.16      2997



/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

#### Above we can see even after equalizing datapoints we are not getting good results , still we will try some more approaches

In [44]:
X_train_flat = np.array([label for sublist in X_train for label in sublist])
X_train_flat = X_train_flat.reshape(-1, 1)
X_train_flat = X_train_flat[:len(y_train)]


svm_classifier = SVC(kernel='rbf')

svm_classifier.fit(X_train_resampled, y_train_resampled)


SVC()

In [45]:
y_pred = svm_classifier.predict(X_test_flat)

# Evaluate the best model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.17517517517517517
Classification Report:
              precision    recall  f1-score   support

           0       0.33      0.00      0.01       465
           1       0.19      0.26      0.22       554
           2       0.38      0.01      0.01       480
           3       0.00      0.00      0.00       465
           4       0.17      0.75      0.28       502
           5       0.00      0.00      0.00       531

    accuracy                           0.18      2997
   macro avg       0.18      0.17      0.09      2997
weighted avg       0.18      0.18      0.09      2997



/home/sagar/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sagar/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sagar/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### SVM also giving 18 % accuracy which is not enough for us for deployment

## Final Conclusion

#### After seeing above approaches we will go with imbalance data for model building

In [1]:
import pandas as pd
labels_df = pd.read_csv("/home/sagar/Bombay Software/Processed_data_with_feature_extracted.csv")
labels_dfx = labels_df[labels_df["features_from_images"].notnull()]

import re

# Define a function to extract and convert numerical values
def extract_numbers(s):
    if isinstance(s, bytes):
        s = s.decode('utf-8')  # Decode bytes-like object to string if needed
    numbers = re.findall(r'\d+\.\d+', s)
    return [float(num) for num in numbers]


# Apply the function to the 'features' column and overwrite it
# labels_dfx["features_from_images"] = labels_dfx["features_from_images"].apply(lambda x: extract_numbers(x[1:-1]))
labels_dfx["features_from_images"] = labels_dfx["features_from_images"].apply(extract_numbers)

/tmp/ipykernel_27737/397893426.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels_dfx["features_from_images"] = labels_dfx["features_from_images"].apply(extract_numbers)


In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report

X = labels_dfx["features_from_images"] # Update with your features
# X = X_train_reshaped

y = labels_dfx["Image_Type"]  # Update with your target variable
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [3]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
ros = RandomOverSampler(random_state=42)
X_train_flat = np.array([label for sublist in X_train for label in sublist])
X_train_flat = X_train_flat.reshape(-1, 1)
X_train_flat = X_train_flat[:len(y_train)]
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_flat, y_train)

In [7]:
import numpy as np
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

knn_classifier = KNeighborsClassifier(n_neighbors=2)

knn_classifier.fit(X_train_resampled, y_train_resampled)


X_test_flat = np.array([label for sublist in X_test for label in sublist])
X_test_flat = X_test_flat.reshape(-1, 1)
X_test_flat = X_test_flat[:len(y_test)]

y_pred = knn_classifier.predict(X_test_flat)

# Evaluate the best model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.3035509736540664
Classification Report:
              precision    recall  f1-score   support

           0       0.09      0.14      0.11       516
           1       0.53      0.48      0.50      2760
           2       0.10      0.11      0.10       481
           3       0.11      0.11      0.11       521
           4       0.09      0.09      0.09       474
           5       0.10      0.10      0.10       486

    accuracy                           0.30      5238
   macro avg       0.17      0.17      0.17      5238
weighted avg       0.33      0.30      0.31      5238



In [5]:
import numpy as np
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

svm_classifier = SVC(kernel='rbf')

svm_classifier.fit(X_train_resampled, y_train_resampled)


SVC()

In [6]:
y_pred = svm_classifier.predict(X_test_flat)

# Evaluate the best model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.09851088201603665
Classification Report:
              precision    recall  f1-score   support

           0       0.09      0.25      0.14       516
           1       0.67      0.00      0.00      2760
           2       0.07      0.02      0.03       481
           3       0.10      0.51      0.16       521
           4       0.11      0.21      0.14       474
           5       0.11      0.03      0.05       486

    accuracy                           0.10      5238
   macro avg       0.19      0.17      0.09      5238
weighted avg       0.40      0.10      0.05      5238



In [7]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report

X = labels_dfx["features_from_images"] # Update with your features
# X = X_train_reshaped

y = labels_dfx["Image_Type"]  # Update with your target variable
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
X_train_flat = np.array([label for sublist in X_train for label in sublist])
X_train_flat = X_train_flat.reshape(-1, 1)
X_train_flat = X_train_flat[:len(y_train)]

In [9]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report

rf_classifier = RandomForestClassifier()

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Perform grid search with 5-fold cross-validation
grid_search = GridSearchCV(rf_classifier, param_grid, cv=5, n_jobs=-1, verbose=1)

X_train_flat = X_train_flat[:len(y_train)]
grid_search.fit(X_train_flat, y_train)

# Get the best hyperparameters from grid search
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)
# Fit the grid search to the training data
print("Best Hyperparameters:", best_params)


Fitting 5 folds for each of 243 candidates, totalling 1215 fits


/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

...

/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Best Hyperparameters: {'max_depth': 10, 'max_features': 'auto', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 300}
Best Hyperparameters: {'max_depth': 10, 'max_features': 'auto', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 300}


In [10]:
best_rf_model = grid_search.best_estimator_
X_test_flat = np.array([label for sublist in X_test for label in sublist])
X_test_flat = X_test_flat.reshape(-1, 1)
X_test_flat = X_test_flat[:len(y_test)]

y_pred = best_rf_model.predict(X_test_flat)

# Evaluate the best model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.5269186712485682
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       516
           1       0.53      1.00      0.69      2760
           2       0.00      0.00      0.00       481
           3       0.00      0.00      0.00       521
           4       0.00      0.00      0.00       474
           5       0.00      0.00      0.00       486

    accuracy                           0.53      5238
   macro avg       0.09      0.17      0.12      5238
weighted avg       0.28      0.53      0.36      5238



/home/sagar/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sagar/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sagar/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
import joblib

# Save the best model obtained from grid search
joblib.dump(best_rf_model, 'best_random_forest_modelx_new.pkl')
joblib.dump(label_encoder, 'label_encoder_newx.pkl')


['label_encoder_newx.pkl']

/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/sagar/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

### By above all approaches we found that random forest with imbalance data using Smote technique and with Grid search hyperparameter tuning is giving 53 % accuracy for our problem statement , due to our constraint we will use this as final model for deployment

##### Note:- Cell numbers may not be in sequence because I have tried various way of solving problem, again i cannot run all of them because of time limitation for submission of assignment

#### Future enhancement we can do

#### We can upsample datapoints for all remaining 5 classes so that it will equally distributed
#### We can use advance Computer vision techniques to make model more powerfull
#### We can use other algorithms for enhancement of models and predictions